In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.metrics import *
import tensorflow as tf
from tensorflow import keras

In [2]:
os.chdir("/home/t326h379/OGP")

In [3]:
df_negative_test = pd.read_csv('Feature_Extraction_July_30_Sprint_Gly_Negative_Independent_Test_3376_.txt',header=None)
df_positive_test = pd.read_csv('Feature_Extraction_July_30_Sprint_Gly_Positive_Independent_Test_79_.txt',header=None)

In [4]:
Header_name = ["Position","PID","S or T"]

col_of_feature = [i for i in range(1,1025)]

Header_name = Header_name + col_of_feature

df_positive_test.columns = Header_name
df_negative_test.columns = Header_name

# Load the Trained Model

In [5]:
model = tf.keras.models.load_model("Compare_with_Sprint_Gly486__103928619___.h5")

In [6]:
model.summary()

Model: "sequential_786"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1572 (Dense)           (None, 512)               524800    
_________________________________________________________________
dropout_2358 (Dropout)       (None, 512)               0         
_________________________________________________________________
dense_1573 (Dense)           (None, 256)               131328    
_________________________________________________________________
dropout_2359 (Dropout)       (None, 256)               0         
_________________________________________________________________
Dense_1 (Dense)              (None, 32)                8224      
_________________________________________________________________
dropout_2360 (Dropout)       (None, 32)                0         
_________________________________________________________________
Dense_2 (Dense)              (None, 2)              

In [7]:
frames_test = [df_positive_test, df_negative_test]

O_linked_testing = pd.concat(frames_test,ignore_index = True)

df_Test_array = O_linked_testing.drop(["Position","PID","S or T"],axis=1)
df_Test_array = np.array(df_Test_array)

X_test_full = df_Test_array

y_test_full = np.array([1]*79+[0]*3376)

In [8]:
X_independent = X_test_full
y_independent = y_test_full

In [9]:
Y_pred = model.predict(X_independent)
Y_pred = (Y_pred > 0.5)
y_pred = [np.argmax(y, axis=None, out=None) for y in Y_pred]
y_pred = np.array(y_pred)

confusion = confusion_matrix(y_independent,y_pred)
print()
print("Matthews Correlation : ",matthews_corrcoef(y_independent, y_pred))
print()
print("Confusion Matrix : \n",confusion_matrix(y_independent, y_pred))
print()
print("Accuracy on test set:   ",accuracy_score(y_independent, y_pred))

cm = confusion_matrix(y_independent, y_pred)

TP = cm[1][1]
TN = cm[0][0]
FP = cm[0][1]
FN = cm[1][0]

mcc = matthews_corrcoef(y_independent, y_pred)

Sensitivity = TP/(TP+FN)

Specificity = TN/(TN+FP)

Precision = TP/(TP+FP)

print()
print("Sensitivity:   ",Sensitivity,"\t","Specificity:   ",Specificity)
print()
print("Precision  :",Precision)
print()
print(classification_report(y_independent, y_pred))

fpr, tpr, _ = roc_curve(y_independent, y_pred)

roc_auc_test = auc(fpr,tpr)
print()

print("Area Under Curve:   ",roc_auc_test)


Matthews Correlation :  0.29761701839122134

Confusion Matrix : 
 [[2892  484]
 [  10   69]]

Accuracy on test set:    0.8570188133140376

Sensitivity:    0.8734177215189873 	 Specificity:    0.8566350710900474

Precision  : 0.12477396021699819

              precision    recall  f1-score   support

           0       1.00      0.86      0.92      3376
           1       0.12      0.87      0.22        79

    accuracy                           0.86      3455
   macro avg       0.56      0.87      0.57      3455
weighted avg       0.98      0.86      0.91      3455


Area Under Curve:    0.8650263963045175


# Thank You